In [1]:
# -----------------
# Setup & config
# -----------------
import warnings, os
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, accuracy_score, brier_score_loss, log_loss


In [2]:
# -----------------
# Speed/size knobs 
# -----------------
CSV_PATH           = "adjusted_dataset.csv"
TOP_N_COMPANIES    = 10      # analyze only the most-populated Companies for speed
MIN_ROWS_PER_CO    = 60      # skip tiny time series
N_SPLITS           = 3       # per-company TimeSeriesSplit folds
N_TREES            = 100     # RF trees (keep small for speed)
MAX_DEPTH          = 6       # RF max depth
ALPHA              = 0.05    # significance level for tests
COST_PER_TURN      = 0.0005  # 5 bps turnover cost in backtest
WRITE_CSVS         = False   # set True to save outputs


In [3]:
# ------------------------
# 1) Load & Explore Data
# ------------------------
assert os.path.exists(CSV_PATH), f"Can't find {CSV_PATH}"
df = pd.read_csv(CSV_PATH, parse_dates=["Date"])
df = df.rename(columns=lambda c: c.strip().replace(" ", "_"))
if "Adj_Close" in df.columns: df = df.rename(columns={"Adj_Close":"AdjClose"})
if "Adj Close" in df.columns: df = df.rename(columns={"Adj Close":"AdjClose"})
if "Close" in df.columns and "AdjClose" not in df.columns:
    df = df.rename(columns={"Close":"AdjClose"})

required = {"Company","Date","Open","AdjClose"}
missing = required - set(df.columns)
assert not missing, f"Missing required columns: {missing}"

df = df.dropna(subset=list(required)).copy()
df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.tz_localize(None)
df = df[(df["Open"]>0) & (df["AdjClose"]>0)].copy()
df = df.sort_values(["Company","Date"]).reset_index(drop=True)

# focus on top-N companies to keep it fast
top_companies = df["Company"].value_counts().nlargest(TOP_N_COMPANIES).index
df = df[df["Company"].isin(top_companies)].copy()


In [4]:
# -----------------------
# 2) Features (NO leakage)
# -----------------------
def add_features(g):
    g = g.sort_values("Date").copy()
    g["prev_adj"]      = g["AdjClose"].shift(1)
    g["daily_ret"]     = (g["AdjClose"] - g["prev_adj"]) / g["prev_adj"]         # close→close
    g["intraday_ret"]  = (g["AdjClose"] - g["Open"])     / g["Open"]             # open→close
    g["overnight_gap"] = (g["Open"]      - g["prev_adj"]) / g["prev_adj"]        # prev close→open

    if "Volume" in g.columns:
        roll = g["Volume"].rolling(20, min_periods=5)
        g["vol_roll_mean"] = roll.mean()
        g["vol_roll_std"]  = roll.std()
        g["vol_zscore"]    = (g["Volume"] - g["vol_roll_mean"]) / g["vol_roll_std"]
        g["vol_spike"]     = (g["vol_zscore"] > 2).astype(int)
    else:
        g["vol_zscore"] = np.nan
        g["vol_spike"]  = 0

    # anti-leak: only use volume info known BEFORE today's intraday move
    g["vol_zscore_lag1"] = g["vol_zscore"].shift(1)
    g["vol_spike_lag1"]  = g["vol_spike"].shift(1).fillna(0).astype(int)

    g["weekday"]    = g["Date"].dt.day_name()
    g["month"]      = g["Date"].dt.month_name()
    g["target_up"]  = (g["intraday_ret"] > 0).astype(int)
    return g

df = df.groupby("Company", group_keys=False).apply(add_features).reset_index(drop=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=["daily_ret","intraday_ret","overnight_gap"]).copy()




In [5]:
# ======================================================================
# 3) RQ1 — Similar return profiles? (variance/tails)
# ======================================================================
rq1_summary = (df.groupby("Company")["daily_ret"]
                 .agg(count="count", mean="mean", var="var", std="std")
                 .assign(
                     kurtosis = df.groupby("Company")["daily_ret"].apply(pd.Series.kurtosis),
                     pct_extreme = df.groupby("Company")["daily_ret"].apply(
                         lambda x: (x.abs() > (3*x.std())).mean()*100
                     ))
               .sort_values("var", ascending=False))

groups = [g["daily_ret"].values for _, g in df.groupby("Company") if len(g)>=MIN_ROWS_PER_CO]
rq1_levene_p  = stats.levene(*groups, center="median").pvalue if len(groups)>=2 else np.nan
rq1_fligner_p = stats.fligner(*groups).pvalue if len(groups)>=2 else np.nan
rq1_reject = ((not np.isnan(rq1_levene_p) and rq1_levene_p<ALPHA) or
              (not np.isnan(rq1_fligner_p) and rq1_fligner_p<ALPHA))


In [6]:
# ======================================================================
# 4) RQ2 — Gaps → intraday direction (association + prediction)
# ======================================================================
def sign_label(x): return "pos" if x>0 else ("neg" if x<0 else "zero")
df["gap_sign"] = df["overnight_gap"].apply(sign_label)
ct = pd.crosstab(df["gap_sign"], df["target_up"])
rq2_chi2_p = stats.chi2_contingency(ct)[1] if ct.values.sum()>0 else np.nan

# Logit (pooled, inference; uses lagged volume to avoid leakage)
logit_df = df.dropna(subset=["target_up","overnight_gap","vol_zscore_lag1"]).copy()
logit = smf.logit("target_up ~ overnight_gap + vol_zscore_lag1", data=logit_df).fit(disp=False)
rq2_logit_params = logit.params.copy()
rq2_logit_pvals  = logit.pvalues.copy()
rq2_logit_or     = np.exp(rq2_logit_params)
rq2_support_assoc = (rq2_logit_pvals.get("overnight_gap",1.0) < ALPHA) and (rq2_logit_params.get("overnight_gap",0) > 0)

# ---------- RQ2 ML: per-company, EXPLICIT TimeSeriesSplit ----------
features = ["overnight_gap","vol_zscore_lag1"]
def eval_company_ml(g):
    g = g.dropna(subset=features+["target_up"]).copy()
    if len(g) < MIN_ROWS_PER_CO:
        return pd.Series({"AUC":np.nan,"ACC":np.nan,"Brier":np.nan,"LogLoss":np.nan,"n":len(g)})

    X = g[features].values
    y = g["target_up"].values
    n_splits = min(N_SPLITS, max(2, len(g)//MIN_ROWS_PER_CO))
    tscv = TimeSeriesSplit(n_splits=n_splits)

    aucs, accs, briers, logs = [], [], [], []
    rf = RandomForestClassifier(n_estimators=N_TREES, max_depth=MAX_DEPTH, random_state=42)

    for tr_idx, te_idx in tscv.split(X):
        # explicit splits
        X_train, X_test = X[tr_idx], X[te_idx]
        y_train, y_test = y[tr_idx], y[te_idx]

        rf.fit(X_train, y_train)
        p = rf.predict_proba(X_test)[:,1]
        preds = (p>=0.5).astype(int)

        aucs.append(roc_auc_score(y_test, p))
        accs.append(accuracy_score(y_test, preds))
        briers.append(brier_score_loss(y_test, p))
        logs.append(log_loss(y_test, np.clip(p,1e-6,1-1e-6)))

    return pd.Series({
        "AUC":   np.mean(aucs),
        "ACC":   np.mean(accs),
        "Brier": np.mean(briers),
        "LogLoss": np.mean(logs),
        "n": len(g)
    })

eval_by_co = df.groupby("Company", group_keys=False).apply(eval_company_ml)
ml_summary = eval_by_co[["AUC","ACC","Brier","LogLoss"]].describe().T



In [7]:
# ======================================================================
# 5) RQ3 — (Lagged) volume vs. move size
# ======================================================================
rq3_df = df.dropna(subset=["intraday_ret","vol_zscore_lag1"]).copy()
rq3_df["abs_intraday"] = rq3_df["intraday_ret"].abs()

ols = smf.ols("abs_intraday ~ vol_zscore_lag1", data=rq3_df).fit()
rq3_beta = ols.params.get("vol_zscore_lag1", np.nan)
rq3_pval = ols.pvalues.get("vol_zscore_lag1", np.nan)

spike = rq3_df[rq3_df["vol_spike_lag1"]==1]["abs_intraday"]
normal = rq3_df[rq3_df["vol_spike_lag1"]==0]["abs_intraday"]
rq3_mw_p = stats.mannwhitneyu(spike, normal, alternative="two-sided").pvalue \
           if (len(spike)>=10 and len(normal)>=10) else np.nan
rq3_reject = ((rq3_pval<ALPHA and rq3_beta>0) or (not np.isnan(rq3_mw_p) and rq3_mw_p<ALPHA))



In [8]:
# ======================================================================
# 6) RQ4 — Seasonality (weekday/month), normalized by company
# ======================================================================
def z_by_company(s):
    sd = s.std(ddof=0)
    return (s - s.mean()) / (sd if sd>0 else 1.0)
df["daily_ret_z"] = df.groupby("Company")["daily_ret"].transform(z_by_company)

wk_groups = [g["daily_ret_z"].dropna().values for _, g in df.groupby("weekday") if len(g)>=30]
rq4_kw_weekday_p = stats.kruskal(*wk_groups).pvalue if len(wk_groups)>=2 else np.nan

mo_groups = [g["daily_ret_z"].dropna().values for _, g in df.groupby("month") if len(g)>=30]
rq4_kw_month_p = stats.kruskal(*mo_groups).pvalue if len(mo_groups)>=2 else np.nan

rq4_reject = ((not np.isnan(rq4_kw_weekday_p) and rq4_kw_weekday_p<ALPHA) or
              (not np.isnan(rq4_kw_month_p)   and rq4_kw_month_p  <ALPHA))



In [9]:
# ======================================================================
# 7) 80/20 chronological HOLDOUT for RQ2 prediction (global + per-company)
# ======================================================================
needed_cols = features + ["target_up","Date","Company"]
hold_df = df.dropna(subset=needed_cols).copy().sort_values(["Date","Company"])
cutoff_date = hold_df["Date"].quantile(0.80)
train = hold_df[hold_df["Date"] <= cutoff_date].copy()
test  = hold_df[hold_df["Date"] >  cutoff_date].copy()

# Global Logit
g_logit = smf.logit("target_up ~ overnight_gap + vol_zscore_lag1", data=train).fit(disp=False)
p_logit  = g_logit.predict(test[features])
pred_l   = (p_logit>=0.5).astype(int)
logit_auc = roc_auc_score(test["target_up"], p_logit)
logit_acc = accuracy_score(test["target_up"], pred_l)
logit_brier = brier_score_loss(test["target_up"], p_logit)
logit_ll = log_loss(test["target_up"], np.clip(p_logit,1e-6,1-1e-6))

# Global RF
rf_global = RandomForestClassifier(n_estimators=200, random_state=42)
rf_global.fit(train[features], train["target_up"])
p_rf  = rf_global.predict_proba(test[features])[:,1]
pred_r= (p_rf>=0.5).astype(int)
rf_auc = roc_auc_score(test["target_up"], p_rf)
rf_acc = accuracy_score(test["target_up"], pred_r)
rf_brier = brier_score_loss(test["target_up"], p_rf)
rf_ll = log_loss(test["target_up"], np.clip(p_rf,1e-6,1-1e-6))

# Per-company 80/20 (chronological) summary
def per_company_holdout(g):
    g = g.dropna(subset=needed_cols).sort_values("Date")
    n = len(g)
    if n < MIN_ROWS_PER_CO: 
        return pd.Series({"AUC_Logit":np.nan,"ACC_Logit":np.nan,"AUC_RF":np.nan,"ACC_RF":np.nan,"n":n})
    cut = g["Date"].iloc[int(np.floor(0.8*n))]
    tr, te = g[g["Date"]<=cut], g[g["Date"]>cut]
    if len(te)<10 or len(tr)<30:
        return pd.Series({"AUC_Logit":np.nan,"ACC_Logit":np.nan,"AUC_RF":np.nan,"ACC_RF":np.nan,"n":n})
    # Logit
    try:
        m = smf.logit("target_up ~ overnight_gap + vol_zscore_lag1", data=tr).fit(disp=False)
        p = m.predict(te[features])
        auc_l = roc_auc_score(te["target_up"], p)
        acc_l = accuracy_score(te["target_up"], (p>=0.5).astype(int))
    except Exception:
        auc_l, acc_l = np.nan, np.nan
    # RF
    try:
        r = RandomForestClassifier(n_estimators=200, random_state=42)
        r.fit(tr[features], tr["target_up"])
        p2 = r.predict_proba(te[features])[:, 1]
        auc_r = roc_auc_score(te["target_up"], p2)
        acc_r = accuracy_score(te["target_up"], (p2>=0.5).astype(int))
    except Exception:
        auc_r, acc_r = np.nan, np.nan
    return pd.Series({"AUC_Logit":auc_l,"ACC_Logit":acc_l,"AUC_RF":auc_r,"ACC_RF":acc_r,"n":n})

perco_eval = hold_df.groupby("Company", group_keys=False).apply(per_company_holdout)
perco_med  = perco_eval[["AUC_Logit","ACC_Logit","AUC_RF","ACC_RF","n"]].median(numeric_only=True)



In [10]:
# ======================================================================
# 8) Portfolio-correct baseline backtest (naive gap rule)
# ======================================================================
bt = df.sort_values(["Company","Date"]).copy()
bt["side_rule"] = (bt["overnight_gap"] > 0).astype(int)
bt["ret_rule"]  = np.where(bt["side_rule"]==1, bt["intraday_ret"], -bt["intraday_ret"])
bt["turnover"]  = bt.groupby("Company")["side_rule"].diff().abs().fillna(0)
by_day = bt.groupby("Date", as_index=True)
daily_ret   = by_day["ret_rule"].mean().fillna(0)
daily_costs = (by_day["turnover"].mean() * COST_PER_TURN).fillna(0)
net_daily   = (daily_ret - daily_costs).fillna(0)
equity = (1+net_daily).cumprod()
def CAGR(eq, ann=252): return (eq.iloc[-1])**(ann/len(eq)) - 1 if len(eq)>0 else np.nan
def max_dd(eq): return (eq/eq.cummax()-1).min() if len(eq)>0 else np.nan
bt_cagr, bt_sharpe, bt_mdd = CAGR(equity), (np.sqrt(252)*net_daily.mean()/net_daily.std() if net_daily.std()>0 else np.nan), max_dd(equity)




In [11]:
# ======================================================================
# 9) Decision-focused report: Accept/Reject each H0 + conclusion
# ======================================================================
def yn(b): return "YES" if b else "NO"

print("\n" + "="*90)
print("RESULTS & DECISIONS (α = 0.05)")
print("="*90)

# RQ1
print("\n[RQ1] Similar return profiles?")
print(f"  Levene p = {rq1_levene_p:.4g}" if not np.isnan(rq1_levene_p) else "  Levene: N/A")
print(f"  Fligner p = {rq1_fligner_p:.4g}" if not np.isnan(rq1_fligner_p) else "  Fligner: N/A")
print(f"  Decision: Reject H0? {yn(rq1_reject)}")
if not rq1_summary.empty:
    print("  Top 5 by variance (var, kurtosis, %>|3σ|):")
    print(rq1_summary[["var","kurtosis","pct_extreme"]].head(5), "\n")

# RQ2
print("[RQ2] Do overnight gaps predict intraday direction?")
print(f"  Chi-square p (gap_sign vs up): {rq2_chi2_p:.4g}" if not np.isnan(rq2_chi2_p) else "  Chi-square: N/A")
print("  Logit coefficients (pooled, leak-safe):")
for k in rq2_logit_params.index:
    print(f"    {k:>18}: beta={rq2_logit_params[k]: .4f}, OR={rq2_logit_or[k]: .3f}, p={rq2_logit_pvals[k]:.4g}")
print(f"  Assoc. Decision (Logit 'overnight_gap' >0 & p<α): {yn(rq2_support_assoc)}")

print("\n  ML (per-company TimeSeriesSplit, explicit train/test) summary:")
print(ml_summary[["count","mean","std","min","50%","max"]])

print("\n  80/20 GLOBAL holdout:")
print(f"    LOGIT   AUC={logit_auc:.3f}  ACC={logit_acc:.3f}  Brier={logit_brier:.4f}  LogLoss={logit_ll:.4f}")
print(f"    RF      AUC={rf_auc:.3f}     ACC={rf_acc:.3f}     Brier={rf_brier:.4f}     LogLoss={rf_ll:.4f}")
print("  80/20 PER-COMPANY holdout (medians across tickers):")
print(perco_med)

# Decide RQ2 predictability: both statistical association AND some predictive edge
rq2_predictive = (rq2_support_assoc and (rf_auc >= 0.55 or logit_auc >= 0.55))
print(f"\n  Predictive Decision: Reject H0? {yn(rq2_predictive)}")

# RQ3
print("\n[RQ3] Do (lagged) volume spikes lead to larger moves?")
print(f"  OLS beta(vol_zscore_lag1)={rq3_beta:.4f}, p={rq3_pval:.4g}")
print(f"  Mann–Whitney p (spike_prev vs none) = {rq3_mw_p:.4g}" if not np.isnan(rq3_mw_p) else "  Mann–Whitney: N/A")
print(f"  Decision: Reject H0? {yn(rq3_reject)}")

# RQ4
print("\n[RQ4] Seasonality (weekday/month) after per-company normalization:")
print(f"  Weekday Kruskal–Wallis p = {rq4_kw_weekday_p:.4g}" if not np.isnan(rq4_kw_weekday_p) else "  Weekday: N/A")
print(f"  Month   Kruskal–Wallis p = {rq4_kw_month_p:.4g}"   if not np.isnan(rq4_kw_month_p)   else "  Month: N/A")
print(f"  Decision: Reject H0? {yn(rq4_reject)}")

# Backtest
def sharpe(daily):
    sd = daily.std()
    return np.sqrt(252)*daily.mean()/sd if sd>0 else np.nan
print("\n[Baseline backtest] Naive equal-weight gap rule (incl. simple costs)")
print(f"  CAGR={CAGR(equity):.2%}  Sharpe={sharpe(net_daily):.2f}  MaxDD={max_dd(equity):.2%}")




RESULTS & DECISIONS (α = 0.05)

[RQ1] Similar return profiles?
  Levene p = 7.648e-57
  Fligner p = 5.768e-161
  Decision: Reject H0? YES
  Top 5 by variance (var, kurtosis, %>|3σ|):
              var    kurtosis  pct_extreme
Company                                   
ZS       0.005535  252.781354     0.795545
ADBE     0.001704  129.191945     0.795545
AAPL     0.001414  196.285301     0.636436
ABEV     0.001080   39.296411     1.113763
ACGL     0.001047  119.978017     1.272872 

[RQ2] Do overnight gaps predict intraday direction?
  Chi-square p (gap_sign vs up): 0.1517
  Logit coefficients (pooled, leak-safe):
             Intercept: beta= 0.1036, OR= 1.109, p=7.011e-09
         overnight_gap: beta=-0.8598, OR= 0.423, p=0.1254
       vol_zscore_lag1: beta= 0.0126, OR= 1.013, p=0.461
  Assoc. Decision (Logit 'overnight_gap' >0 & p<α): NO

  ML (per-company TimeSeriesSplit, explicit train/test) summary:
         count      mean       std       min       50%       max
AUC       10.0  0

In [12]:
# Overall project conclusion
print("\n" + "-"*90)
conclusions = []
if rq1_reject:
    conclusions.append("Return profiles differ across companies—tail risk/variance is not uniform.")
else:
    conclusions.append("No strong evidence that company return profiles materially differ in variance/tails.")

if rq2_predictive:
    conclusions.append("Overnight gaps provide statistically significant AND modestly predictive information for intraday direction.")
else:
    conclusions.append("Overnight gaps show limited/mixed predictive power across the holdout and companies.")

if rq3_reject:
    conclusions.append("Higher prior volume is associated with larger subsequent intraday moves.")
else:
    conclusions.append("No robust link between prior-day volume spikes and next-day move size was detected.")

if rq4_reject:
    conclusions.append("Some seasonal (weekday/month) effects exist after normalizing by company.")
else:
    conclusions.append("No compelling aggregate seasonal effects detected.")

conclusions.append("Baseline, cost-aware gap rule shows baseline-level performance; more signal engineering and tuning are needed for stronger economics.")

print("FINAL CONCLUSION:")
for i, c in enumerate(conclusions, 1):
    print(f"  {i}. {c}")
print("-"*90)


------------------------------------------------------------------------------------------
FINAL CONCLUSION:
  1. Return profiles differ across companies—tail risk/variance is not uniform.
  2. Overnight gaps show limited/mixed predictive power across the holdout and companies.
  3. Higher prior volume is associated with larger subsequent intraday moves.
  4. Some seasonal (weekday/month) effects exist after normalizing by company.
  5. Baseline, cost-aware gap rule shows baseline-level performance; more signal engineering and tuning are needed for stronger economics.
------------------------------------------------------------------------------------------
